In [2]:
from collections import deque

https://www.acmicpc.net/problem/1697

### 1697
수빈이는 동생과 숨바꼭질을 하고 있다. 수빈이는 현재 점 N(0 ≤ N ≤ 100,000)에 있고, 동생은 점 K(0 ≤ K ≤ 100,000)에 있다. 수빈이는 걷거나 순간이동을 할 수 있다. 만약, 수빈이의 위치가 X일 때 걷는다면 1초 후에 X-1 또는 X+1로 이동하게 된다. 순간이동을 하는 경우에는 1초 후에 2*X의 위치로 이동하게 된다.

수빈이와 동생의 위치가 주어졌을 때, **수빈이가 동생을 찾을 수 있는 가장 빠른 시간**이 몇 초 후인지 구하는 프로그램을 작성하시오.

**입력** 첫 번째 줄에 수빈이가 있는 위치 N과 동생이 있는 위치 K가 주어진다. N과 K는 정수이다.

**출력**
수빈이가 동생을 찾는 가장 빠른 시간을 출력한다.

### 문제 해석
- 수빈이가 동생을 찾을 수 있는 가장 빠른 시간 = 수빈노드가 동생노드로 가는 가장 빠른 경로에서 노드 개수*시간(1초)
- 5에서 17로 가는 방법
    - X=5 -> (걷기) X=6 or 4 -> X=7 or 5...
    - x=5 -> (순간이동) x=10 -> x = 20...
- 그래프를 나타내면, 
    - graph[5] = [4, 10]
    - graph[6] = [5, 12]
    
- start_node = 수빈이의 위치 (N)

### 예시 해석
- 5 -> 17로 갈 때, 어떻게 하면 4번만에 갈 수 있을까?
    - 5 -> 4 -> 8 -> 16 -> 17 
- 이걸 어떻게 bfs로 구현할 수 있을까?

# Note
**shallow copy and deep copy**

다음과 같이 graph를 만들었을 때, 문제가 발생!!

In [42]:
N, K = 5, 17
ans = 4

graph = [[]]*(N) #0부터 시작
for i in range(N):
    next_position = [i-1, i+1, 2*i]
    graph[i].extend(next_position) # 걷기, 순간이동
    break
visited = [False]*(K)

graph의 천번째 인덱스에만 next_position을 extend 하고 break했는데, 왜 결과는 전체 인덱스에 다 extend되었을까? <br>

In [43]:
graph

[[-1, 1, 0], [-1, 1, 0], [-1, 1, 0], [-1, 1, 0], [-1, 1, 0]]

느낌이 쎄해서 바로 주소값을 찍어보았다..<br>
파이썬의 nested list는 shallow copy라는 것을 까먹고 있었다..

In [36]:
id(graph[0]), id(graph[1])

(140620724629888, 140620724629888)

graph를 빈 리스트로 만들고, 새로운 next position 리스트가 생길때마다 append를 해주었다.

In [54]:
N, K = 5, 17
ans = 4

graph = []
for i in range(K):
    next_position = [i-1, i+1, 2*i]
    graph.append(next_position) # 걷기, 순간이동
    break
visited = [False]*(K)

In [55]:
graph

[[-1, 1, 0]]

# Solution

### trial1

- 처음에 접근을 잘못했다. 갈 수있는 경로들의 최단 거리를 구해야 되는데, 그냥 무작정 BFS만 돌려버렸으니...
- 최단 경로라는 것을 알기 위해, 15에 도달할 때까지의 경로들의 시간을 측정해야 했다.
- 다음의 블로그를 참고했다.
https://chanhuiseok.github.io/posts/baek-14/

<u>큐에 처음 들어갈 때 그 위치에 도달했을 때 걸린 시간도 같이 넣어주기 위해 pair<int, int> 형 큐를 선언</u>


In [37]:
def bfs(graph, node, visited, time):
    to_visit = deque([(node, time)]) # 시작노드를 방문할 큐에 넣어준다
    visited[node] = True # 방문한다
    
    while len(to_visit) > 0:
        v, t = to_visit.popleft() # 방문할 큐에서 노드를 하나 뺀다
        print((v,t), end=' -> ')
        for i in graph[v]: #현재 방문한 노드 기준 다음 노드를 확인한다
            if i == K: # K = len(visited) = 17
                return t+1
            if i < len(visited) and not visited[i]: # 다음 노드를 방문하지 않았다면,
                to_visit.append((i, t+1))
                visited[v] = True # 방문해준다
    return arrived_time

In [183]:
N, K = 5, 17
ans = 4

graph = []
for i in range(K):
    next_position = [i-1, i+1, 2*i]
    graph.append(next_position) # 걷기, 순간이동
visited = [False]*(K)

time = 0
start = N
res = bfs(graph, start, visited, time)

(5, 0) -> (4, 1) -> (6, 1) -> (10, 1) -> (3, 2) -> (8, 2) -> (7, 2) -> (12, 2) -> (9, 2) -> (11, 2) -> (2, 3) -> (7, 3) -> (9, 3) -> (16, 3) -> 

In [184]:
res

4

In [185]:
for i in range(len(graph)):
    print(i, ':', graph[i])

0 : [-1, 1, 0]
1 : [0, 2, 2]
2 : [1, 3, 4]
3 : [2, 4, 6]
4 : [3, 5, 8]
5 : [4, 6, 10]
6 : [5, 7, 12]
7 : [6, 8, 14]
8 : [7, 9, 16]
9 : [8, 10, 18]
10 : [9, 11, 20]
11 : [10, 12, 22]
12 : [11, 13, 24]
13 : [12, 14, 26]
14 : [13, 15, 28]
15 : [14, 16, 30]
16 : [15, 17, 32]


### trial 2
- index error가 떠버렸다
- 다른 테스트 케이스에서 반례가 나와서 다시 고친다..

https://www.acmicpc.net/board/view/32486

**테스트 케이스**

4 -> 7로 가는 최단 경로와 시간:

4 -> 8 -> 7 , 2초

In [13]:
from collections import deque


N, K = map(int, input().split())

graph = []
for i in range(K):
    next_position = [i-1, i+1, 2*i]
    graph.append(next_position) # 걷기, 순간이동
visited = [False]*(K)

time = 0
start = N
res = bfs(graph, start, visited, time)
print(res)

4 7
(4, 0) -> (3, 1) -> (5, 1) -> (2, 2) -> (6, 2) -> 3


문제: 4 -> [3, 5, 8] 까지 방문하지 않는다.. 왜?

현재 코드에서는 도착 위치보다 작은 지점만 방문하도록 코딩되어있기 때문에, 7보다 큰 8은 방문하지 않았던 것.. -> 여기를 고치자!

In [15]:
graph

[[-1, 1, 0],
 [0, 2, 2],
 [1, 3, 4],
 [2, 4, 6],
 [3, 5, 8],
 [4, 6, 10],
 [5, 7, 12]]

### trial 3
- 틀렸다고 해서 테스트 케이스를 돌려본다..
- 제자리 걸음, 왼쪽으로 가는 경우를 생각하지 못했다
    - 0 -> 0 
    - 1000 -> 0
- 메모리가 터지는 경우도 있다고 하는데, 아마 탐색범위 설정을 미리 해줘서 나는 통과했다

**최종 코드**

In [77]:
def bfs(graph, node, visited, time):
    to_visit = deque([(node, time)])
    visited[node] = True 
    
    while len(to_visit) > 0:
        v, t = to_visit.popleft() 
        print((v,t), end=' -> ')
        # 도착한 뒤 리턴하는 부분을 for 문 바깥으로 뺐다 -> 제자리 걸음인 경우를 해결함
        if v == K: 
            return t
        for i in graph[v]:
            if i < len(visited) and not visited[i]:
                to_visit.append((i, t+1))
                visited[v] = True 
            
    return arrived_time

N, K = map(int, input().split())
max_pos = max(N, K) # 왼쪽으로 가는 경우도 고려할 수 있게 함

graph = []
for i in range(max_pos*2+1):
    next_position = [i-1, i+1, 2*i]
    graph.append(next_position) # 걷기, 순간이동
visited = [False]*(max_pos*2+1)

time = 0
start = N
res = bfs(graph, start, visited, time)

1 3
(1, 0) -> (0, 1) -> (2, 1) -> (2, 1) -> (-1, 2) -> (3, 2) -> 

In [78]:
res

2